In [1]:
from ortools.linear_solver import pywraplp
import numpy as np
import pandas as pd

C:\Users\mesek\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\mesek\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


# Example 2
Maximize $7x_1 + 8x_2 + 2x_3 + 9x_4 + 6x_5$ with following constraints:\
$5x_1 + 7x_2 + 9x_3 + 2x_4 + x_5 \leq 250$ \
$18x_1 +4x_2 - 9x_3 + 10x_4 + 12x_5 \leq 285 $ \
$4x_1 + 7x_2 + 3x_3 + 8x_4 + 5x_5 \leq 211$ \
$ 5x_1 +13x_2 + 16x_3 + 3x_4 -7x_5 \leq 312$ \
Where $x_1, x_2,... , x_5$ are non-negative integers ($ \geq 0$)

In [2]:
def create_data_model():
    # Stores data for the problem
    # TO DO: automate it so that is reads constraints from a file or the function takes arguments and assign it to the
    # dictionary
    data = {}
    data["constraint_coeffs"] = [
        [5, 7, 9, 2, 1],
        [18, 4, -9, 10, 12],
        [4, 7, 3, 8, 5],
        [5, 13, 16, 3, -7]
    ]
    data["bounds"] = [250, 285, 211, 315]
    data["obj_coeffs"] = [7, 8, 2, 9, 6] # from the objective function
    data["num_vars"] = 5
    data["num_constraints"] = 4
    return data

In [3]:
data = create_data_model()

In [4]:
# SCIP is a third party Constraint niteger program solver https://www.scipopt.org/
solver = pywraplp.Solver.CreateSolver("SCIP") 

In [5]:
infinity = solver.infinity()
x = {}
for j in range(data["num_vars"]):
    x[j] = solver.IntVar(0, infinity, "x[%i]" % j) # f"x{num_var}"

In [6]:
# Constraints
for i in range(data["num_constraints"]):
    constraint = solver.RowConstraint(0, data["bounds"][i], "") # sets the bound (e.g. <= 250), 1st arg lower bound, 2nd upper
    for j in range(data["num_vars"]):
        constraint.SetCoefficient(x[j], data["constraint_coeffs"][i][j]) # sets the coefficient for each variable in row

In [7]:
solver.NumConstraints()

4

In [8]:
# Define objective
objective = solver.Objective()
for j in range(data["num_vars"]):
    objective.SetCoefficient(x[j], data["obj_coeffs"][j])
objective.SetMaximization()

In [9]:
status = solver.Solve()

In [10]:
if status == pywraplp.Solver.OPTIMAL:
    print(f"Objective value = {solver.Objective().Value()}")
    for j in range(data['num_vars']):
        print(f"x{j} = {x[j].solution_value()}")
    print()
    print(f"Problem solved in {solver.wall_time():d} ms")
    print(f"Problem solved in {solver.iterations():d} iterations")
    print(f"Problem solved in {solver.nodes():d} branch-and-bound-nodes")
else:
    print("The problem does not have an optimal solution.")

Objective value = 259.99999999999966
x0 = 8.0
x1 = 21.0
x2 = 0.0
x3 = 2.0
x4 = 3.0

Problem solved in 7199 ms
Problem solved in 58 iterations
Problem solved in 7 branch-and-bound-nodes


In [11]:
# Array x_ip with solution of integer program
x_ip = np.zeros(5)
for i in range(data['num_vars']):
    x_ip[i] = x[i].solution_value()

# Remarks
The example here: \
https://developers.google.com/optimization/mip/mip_var_array \
yield a different solution which may be achieved by bounding the possible values of variables x to a maximum of 16

# Compare with linear programming solution

In [12]:
data = create_data_model()
solver = pywraplp.Solver.CreateSolver("GLOP")

In [13]:
infinity = solver.infinity()
x = {}
for j in range(data["num_vars"]):
    x[j] = solver.IntVar(0, infinity, "x[%i]" % j) # f"x{num_var}"

In [14]:
# Constraints
for i in range(data["num_constraints"]):
    constraint = solver.RowConstraint(0, data["bounds"][i], "") # sets the bound (e.g. <= 250), 1st arg lower bound, 2nd upper
    for j in range(data["num_vars"]):
        constraint.SetCoefficient(x[j], data["constraint_coeffs"][i][j]) # sets the coefficient for each variable in row

In [15]:
solver.NumConstraints()

4

In [16]:
# Define objective
objective = solver.Objective()
for j in range(data["num_vars"]):
    objective.SetCoefficient(x[j], data["obj_coeffs"][j])
objective.SetMaximization()

In [17]:
status = solver.Solve()

In [18]:
if status == pywraplp.Solver.OPTIMAL:
    print(f"Objective value = {solver.Objective().Value()}")
    for j in range(data['num_vars']):
        print(f"x{j} = {x[j].solution_value()}")
    print()
    print(f"Problem solved in {solver.wall_time():d} ms")
    print(f"Problem solved in {solver.iterations():d} iterations")
    print(f"Problem solved in {solver.nodes():d} branch-and-bound-nodes")
else:
    print("The problem does not have an optimal solution.")

Objective value = 262.6552567237163
x0 = 8.451100244498777
x1 = 22.84290953545232
x2 = 0.0
x3 = 0.0
x4 = 3.459046454767721

Problem solved in 2241 ms
Problem solved in 4 iterations
Problem solved in -1 branch-and-bound-nodes


In [19]:
for i in range(data["num_vars"]):
    print(x[i].solution_value())

8.451100244498777
22.84290953545232
0.0
0.0
3.459046454767721


# Check solutions

In [20]:
def check_solution(solution):
    coeffs = np.array([
        [5, 7, 9, 2, 1],
        [18, 4, -9, 10, 12],
        [4, 7, 3, 8, 5],
        [5, 13, 16, 3, -7]
    ])
    
    data["bounds"] = [250, 285, 211, 315]
    return coeffs@solution

In [21]:
# Array x_lp with solution of linear program
x_lp = np.zeros(5)
for i in range(data['num_vars']):
    x_lp[i] = x[i].solution_value()

In [22]:
check_solution(x_ip)

array([194., 284., 210., 298.])

In [23]:
check_solution(x_lp)

array([205.61491443, 285.        , 211.        , 315.        ])

In [24]:
solutions = {'Integer': x_ip, 'Linear': x_lp}
df = pd.DataFrame(solutions)
df

,Integer,Linear
0,8.0,8.451100
1,21.0,22.842910
2,0.0,0.000000
3,2.0,0.000000
4,3.0,3.459046


# Conclusions

The bounds were 250, 285, 211, 315 and both linear and integer programming solutions fulfilled the constraints. Clearly the linear programming where the solution is a real number doesn't give the optimal solution when we are looking for integer solution. Rounding the numbers of linear programming solution doesn't approximate the integer programming results. Thus, it is required to use packages or software suitable for Integer programming.
